In [1]:
website = "https://www.audible.com/search"


In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.common.exceptions import *

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.webdriver.chrome.service import Service

website = "https://www.audible.com/search"

# Chrome options # Chrome options 
options = Options()
# options.headless = True


## Markdown xpath
1st title = /html/body/div[1]/div[5]/div[5]/div/div[2]/div[4]/div/div/div/span/ul/li[1]/div/div[1]/div/div[2]/div/div/span/ul/li[1]/h3/a
1st title = //*[@id="product-list-a11y-skiplink-target"]/span/ul/li[1]/div/div[1]/div/div[2]/div/div/span/ul/li[1]/h3/a

ul_box = //*[@id="product-list-a11y-skiplink-target"]/span/ul
ul_box = /html/body/div[1]/div[5]/div[5]/div/div[2]/div[4]/div/div/div/span/ul
/html/body/div[1]/div[5]/div[5]/div/div[2]/div[4]/div/div/div/span/ul

In [19]:

# Specify the path to the Chrome driver executable
chrome_driver_path = '/usr/bin/google-chrome'

# Create a Service object
service = Service(chrome_driver_path)

# Set Chrome options if needed
options = webdriver.ChromeOptions()
options.add_argument('--headless')

# Create the Chrome driver instance using the Service object and options
driver = webdriver.Chrome(service=service, options=options)
driver.get(website)
xpath = '//*[@id="product-list-a11y-skiplink-target"]/span/ul'
box = driver.find_element(By.XPATH, xpath)

# true = box.find_element(By.XPATH, './/li[contains(@class, "productListItem")]')  == box.find_element(By.XPATH, f'{xpath}//li[contains(@class, "productListItem")]')
product = box.find_element(By.XPATH, f'{xpath}//li[contains(@class, "productListItem")]')
# print(product.text)
title = product.find_element(By.XPATH, './/h3').text.strip()
author = product.find_element(By.XPATH, './/li[contains(@class, "authorLabel")]').text.split(':')[1].strip()
narrator = product.find_element(By.XPATH, './/li[contains(@class, "narratorLabel")]').text.split(':')[1].strip()
series = product.find_element(By.XPATH, './/li[contains(@class, "seriesLabel")]').text.split(':')[1].strip()
length = product.find_element(By.XPATH, './/li[contains(@class, "runtimeLabel")]').text.split(':')[1].strip()
releaseDate = product.find_element(By.XPATH, './/li[contains(@class, "releaseDateLabel")]').text.split(':')[1].strip()
language = product.find_element(By.XPATH, './/li[contains(@class, "languageLabel")]').text.split(':')[1].strip()

print(title, author, narrator, series, length, releaseDate, language, sep='\n')


# print(title)

driver.close()
driver.quit()


Shattersoul
Kyle Kirrin
Travis Baldree
Ripple System, Book 4
20 hrs and 15 mins
06-02-23
English


In [9]:
print(title)

## scrape the page


In [26]:
from selenium.common.exceptions import NoSuchElementException

# Specify the path to the Chrome driver executable
chrome_driver_path = '/usr/bin/google-chrome'

# Create a Service object
service = Service(chrome_driver_path)

# Set Chrome options if needed
options = webdriver.ChromeOptions()
options.add_argument('--headless')

# Create the Chrome driver instance using the Service object and options
driver = webdriver.Chrome(service=service, options=options)
driver.get(website)

xpath = '//*[@id="product-list-a11y-skiplink-target"]/span/ul'
box = driver.find_element(By.XPATH, xpath)

products = box.find_elements(By.XPATH, f'{xpath}//li[contains(@class, "productListItem")]')

# create a dataframe with the desired info
title = []
author = []
narrator = []
series = []
length = []
releaseDate = []
language = []


for product in products:
    title.append(product.find_element(By.XPATH, './/h3').text.strip())
    author.append(product.find_element(By.XPATH, './/li[contains(@class, "authorLabel")]').text.split(':')[1].strip())

    try:
        narrator.append(product.find_element(By.XPATH, './/li[contains(@class, "narratorLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        narrator.append("N/A")
    
    try:
        series.append(product.find_element(By.XPATH, './/li[contains(@class, "seriesLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        series.append("N/A")
    
    try:
        length.append(product.find_element(By.XPATH, './/li[contains(@class, "runtimeLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        length.append("N/A")
    
    try:
        releaseDate.append(product.find_element(By.XPATH, './/li[contains(@class, "releaseDateLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        releaseDate.append("N/A")
    
    try:
        language.append(product.find_element(By.XPATH, './/li[contains(@class, "languageLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        releaseDate.append("N/A")

        
# create a dataframe 
df = pd.DataFrame({'title':title, 'author':author, 'narrator':narrator,'series':series, "length":length,'releaseDate':releaseDate, 'language':language})

# convert to csv
df.to_csv('audio_books.csv', index=False)

# print(title, author, narrator, series, length, releaseDate, language, sep='\n')


# print(title)

driver.close()
driver.quit()


## Handle Pagination 5 pages

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import *
import pandas as pd

website = "https://www.audible.com/search"

# Specify the path to the Chrome driver executable
chrome_driver_path = '/usr/bin/google-chrome'

# Create a Service object
service = Service(chrome_driver_path)

# Set Chrome options if needed
options = webdriver.ChromeOptions()
options.add_argument('--headless')

# Create the Chrome driver instance using the Service object and options
driver = webdriver.Chrome(service=service, options=options)
driver.get(website)

xpath = '//*[@id="product-list-a11y-skiplink-target"]/span/ul'
box = driver.find_element(By.XPATH, xpath)

products = box.find_elements(By.XPATH, f'{xpath}//li[contains(@class, "productListItem")]')

# create a dataframe with the desired info
title = []
author = []
narrator = []
series = []
length = []
releaseDate = []
language = []


for product in products:
    title.append(product.find_element(By.XPATH, './/h3').text.strip())
    author.append(product.find_element(By.XPATH, './/li[contains(@class, "authorLabel")]').text.split(':')[1].strip())

    try:
        narrator.append(product.find_element(By.XPATH, './/li[contains(@class, "narratorLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        narrator.append("N/A")
    
    try:
        series.append(product.find_element(By.XPATH, './/li[contains(@class, "seriesLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        series.append("N/A")
    
    try:
        length.append(product.find_element(By.XPATH, './/li[contains(@class, "runtimeLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        length.append("N/A")
    
    try:
        releaseDate.append(product.find_element(By.XPATH, './/li[contains(@class, "releaseDateLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        releaseDate.append("N/A")
    
    try:
        language.append(product.find_element(By.XPATH, './/li[contains(@class, "languageLabel")]').text.split(':')[1].strip())
    except NoSuchElementException:
        releaseDate.append("N/A")

        
# create a dataframe 
df = pd.DataFrame({'title':title, 'author':author, 'narrator':narrator,'series':series, "length":length,'releaseDate':releaseDate, 'language':language})

# convert to csv
df.to_csv('audio_books.csv', index=False)
